In [1]:
import os
import re
import ast
import pandas as pd
from pandas_ods_reader import read_ods
#importing two libraries
#re for regular expression
#pandas, our data cleaning tool

#breakdown of file name

if __name__=='__main__':

    path = '../../Downloads/'
    files = os.listdir(path)
    for f in files:
        path_file_name = path+f
        read_file(path_file_name)

# file_name = '02 Lloyd APPLICATION TEMPLATE 2020'
# file_extension = 'xlsx'

# path and file_name
def read_file(path_file):
    df = pd.read_excel (path_file, sheet_name="Applications")
    
    return df

#Drop unnecessary columns
def drop_columns(dataframe):

    drop_column_names = []

    for col in dataframe.columns:
        if re.search("^Unnamed", col, re.IGNORECASE) is None:
            drop_column_names.append(col)
    
    dataframe = dataframe.drop(drop_column_names, axis=1)
    
    if file_extension == "ods":
        new_column_names = []
        for i in range(len(dataframe.columns)):
            new_column_names.append("unnamed_{}".format(i))
        dataframe.columns = new_column_names
        
    new_columns = []
    for col in dataframe.columns:
        col = col.replace(".","_")
        col = col.replace(": ","_")
        new_columns.append(col.lower())
    
    dataframe.columns = new_columns    
    
    return dataframe

df = drop_columns(df)
print(df)


FileNotFoundError: [Errno 2] No such file or directory: '../../Downloads/02 Lloyd APPLICATION TEMPLATE 2020.xlsx'

In [ ]:
df["row_number"] = df.index # assigning rows for individual's grouping of data later
df["unnamed_1"] = df["unnamed_1"].replace(" ","_",regex=True).str.lower()  #converting this to variable-like
df["unnamed_2"] = df["unnamed_2"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN" else None).astype(str)

df["unnamed_4"] = df["unnamed_4"].replace(" ","_",regex=True).str.lower()
df["unnamed_5"] = df["unnamed_5"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

df["unnamed_6"] = df["unnamed_6"].replace(" ","_",regex=True).str.lower()
df["unnamed_7"] = df["unnamed_7"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

df["unnamed_9"] = df["unnamed_9"].replace(" ","_",regex=True).str.lower()
df["unnamed_10"] = df["unnamed_10"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

df["unnamed_20"] = df["unnamed_20"].replace(" ","_",regex=True).str.lower()
df["unnamed_21"] = df["unnamed_21"].apply(lambda x: x if x!="None" and x!="#N/A" and x!="NaN"else None).astype(str)

#combining related data - by category and value
df["Notice_Credit_Decision"] = df["unnamed_1"].astype(str) +" | " +df["unnamed_2"].astype(str)
df["Basic_Info"] = df["unnamed_4"].astype(str) +" | " +df["unnamed_5"].astype(str)
df["MC_Details"] = df["unnamed_6"].astype(str) +" | " +df["unnamed_7"].astype(str)
df["Credit_Info"] = df["unnamed_9"].astype(str) +" | " +df["unnamed_10"].astype(str)
df["DTIR"] = df["unnamed_20"].astype(str) +" | " +df["unnamed_21"].astype(str)
for col in df.columns:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(",","")
    
groupings_list = [df["Notice_Credit_Decision"].to_dict(), df["Basic_Info"].to_dict(), df["MC_Details"].to_dict(), df["Credit_Info"].to_dict(), df["DTIR"].to_dict()]

all_data = []

per_applicant = []
def group_dataset(x):
# def group_dataset(x, dataset, dataset_list):
#     x is a multiple of 26
    y = x/26
    
    applicant_data_set = []
    for group in groupings_list:
        applicant_data={}
        applicant_set_per_group = {k:v for k,v in group.items() if k>=x and k<26*(y+1)}
        applicant_data[x] = {v for v in applicant_set_per_group.values()}
        
        applicant_data_set.append(applicant_data)

    for row_number in applicant_data_set[0].keys():
        combined_data_tuple = []
        for d in applicant_data_set:
            combined_data_tuple.append(d[row_number])
    
    #manipulate combined_data_tuplee
    t = []
    for s in combined_data_tuple:
        s = list(s)
        for r in s:
            t.append(r)
            
    all_data.append(t)

def get_first_occurence(dataset):
    for key, value in dataset[0].items():
        if key%26 == 0:
            group_dataset(key)

get_first_occurence(groupings_list)

print(all_data)

In [ ]:
# for x in all_data_list

all_data_df = pd.DataFrame(all_data)
    
all_data_df["row_number"] = all_data_df.index    

for col in all_data_df.columns:

    if col!="row_number":
        feature_columns  = [ "{col}_category".format(col=col), "{col}_data".format(col=col) ]

        first_df2 = all_data_df[col].str.split(" | ", n = 2, expand = True) 
        all_data_df[feature_columns[0]]= first_df2[0]
        all_data_df[feature_columns[1]]= first_df2[2]
        all_data_df["category"] = first_df2[0]
        all_data_df["data"] = first_df2[2]
#         all_data_df["row_number"] = all_data_df.index
        if col == 0:
            first_df_off = all_data_df[["row_number","0_category" ,"0_data"]]
            first_df_off.columns = ["row_number", "category", "data"]
        else:
            first_df_off = pd.concat([first_df_off[["row_number","category", "data"]], all_data_df[["row_number","category", "data"]]])

        all_data_df = all_data_df.drop(col, axis=1)

first_df_off = first_df_off.mask(first_df_off["category"].eq('None'))
first_df_off = first_df_off.mask(first_df_off["category"].eq('nan'))
first_df_off = first_df_off.mask(first_df_off["category"].eq(' '))
first_df_off = first_df_off.mask(first_df_off["category"].eq('_'))
# first_df_off = first_df_off.dropna(subset=["category"], axis=0)

first_df_off = first_df_off.mask(first_df_off["data"].eq('None'))
first_df_off = first_df_off.mask(first_df_off["data"].eq('nan'))
first_df_off = first_df_off.mask(first_df_off["data"].eq(' '))
first_df_off = first_df_off.mask(first_df_off["data"].eq('_'))

first_df_off.drop_duplicates(subset=["row_number","category"], keep="first", inplace=True)
first_df_off.reset_index().set_index(['row_number'])
# print(first_df_off)
# first_df_off["row_number_2"] = first_df_off.index 

first_df_off = first_df_off.pivot(index="row_number", columns=["category"], values="data")

first_df_off = first_df_off.dropna(how="all", axis=0)
first_df_off = first_df_off.dropna(how="all", axis=1)

#positioning important fields
first_df_off = first_df_off.dropna(subset=["customer_name"], axis=0)
customer_name = first_df_off.pop("customer_name")
age = first_df_off.pop("age")

first_df_off.insert(0, "customer_name", customer_name)
first_df_off.insert(1, "age", age)


print(first_df_off)

# first_df_off = first_df_off[first_df_off["customer_name"]!="nan"]
first_df_off.to_excel(file_name+"_cleaned_forSwit.xlsx")
# print(first_df_off.sort_values(by=['row_number']))
# print(first_df_off.sort_values(by=['row_number']).iloc[278, 1])